In [1]:
import vectorbt as vbt
import pandas as pd
import numpy as np
import yfinance as yf

# Symbols for ETFs
etfs = ['TLT','GLD','XLP','XLK','XLV']  

# Download historical data
data = yf.download(etfs, start="2005-01-01", end="2020-12-31")['Adj Close']
#data.ffill(inplace=True)  # Forward-fill to handle missing values
data.head(5)


[*********************100%%**********************]  5 of 5 completed


Ticker,GLD,TLT,XLK,XLP,XLV
Date,,,,,
2005-01-03,43.020000,48.357395,16.243130,14.010859,21.538517
2005-01-04,42.740002,47.850624,15.940460,13.962294,21.365288
2005-01-05,42.669998,48.106743,15.886132,13.889442,21.329185
2005-01-06,42.150002,48.139450,15.839572,13.956223,21.487988
2005-01-07,41.840000,48.248436,15.870621,14.022989,21.473549


In [2]:
# Generating the Boolean series
last_day_of_month = data.groupby(pd.Grouper(freq='M')).apply(lambda x: x.index.max())
monthly_last_trading = data.index.isin(last_day_of_month)

# Convert to Series
exits = pd.Series(monthly_last_trading, index=data.index)

exits.loc[exits == True]


/var/folders/yh/nnw5blt935v54t02pj41_rjm0000gn/T/ipykernel_75686/3181827276.py:2: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  last_day_of_month = data.groupby(pd.Grouper(freq='M')).apply(lambda x: x.index.max())


Date
2005-01-31    True
2005-02-28    True
2005-03-31    True
2005-04-29    True
2005-05-31    True
              ... 
2020-08-31    True
2020-09-30    True
2020-10-30    True
2020-11-30    True
2020-12-30    True
Length: 192, dtype: bool

In [3]:
# Generate last day of each month for the given period
rebalance_dates = pd.date_range(start=data.index.min(), end=data.index.max(), freq='BM')

# Define entries on the rebalance dates
entries = pd.DataFrame(False, index=data.index, columns=data.columns)

for date in rebalance_dates:
    if date in entries.index:
        entries.loc[date] = True

print(entries.head(5))

# Define exits - in this strategy, there are technically no exits, just rebalances
exits = pd.DataFrame(False, index=data.index, columns=data.columns)
print(exits.head(40))

# Weights for each ETF in the portfolio
weights = np.repeat(1/len(etfs), len(etfs))

# Create a portfolio with rebalancing on specific dates
portfolio = vbt.Portfolio.from_orders(
    close=data,
    size=weights,
    size_type='target_percent', 
    freq='D',
    init_cash=10000
)

rb_pf = vbt.Portfolio.from_orders(
    close=_price,
    size=rb_size,
    size_type='targetpercent',
    group_by='symbol_group',
    cash_sharing=True,
    call_seq='auto'  # important: sell before buy
# Run the backtest
results = portfolio.total_return()

# Output the results
print(results)



/var/folders/yh/nnw5blt935v54t02pj41_rjm0000gn/T/ipykernel_75686/3640964199.py:2: FutureWarning: 'BM' is deprecated and will be removed in a future version, please use 'BME' instead.
  rebalance_dates = pd.date_range(start=data.index.min(), end=data.index.max(), freq='BM')


Ticker        GLD    TLT    XLK    XLP    XLV
Date                                         
2005-01-03  False  False  False  False  False
2005-01-04  False  False  False  False  False
2005-01-05  False  False  False  False  False
2005-01-06  False  False  False  False  False
2005-01-07  False  False  False  False  False
Ticker        GLD    TLT    XLK    XLP    XLV
Date                                         
2005-01-03  False  False  False  False  False
2005-01-04  False  False  False  False  False
2005-01-05  False  False  False  False  False
2005-01-06  False  False  False  False  False
2005-01-07  False  False  False  False  False
2005-01-10  False  False  False  False  False
2005-01-11  False  False  False  False  False
2005-01-12  False  False  False  False  False
2005-01-13  False  False  False  False  False
2005-01-14  False  False  False  False  False
2005-01-18  False  False  False  False  False
2005-01-19  False  False  False  False  False
2005-01-20  False  False  False  F

In [4]:
# Calculate daily returns
daily_returns = portfolio.returns()

# Calculate annual returns
annual_returns = daily_returns.resample('A').apply(lambda x: (x + 1).prod() - 1)

print("Annual Returns:")
print(annual_returns)


Annual Returns:
Ticker           GLD       TLT       XLK       XLP       XLV
Date                                                        
2005-12-31  0.044339  0.009714  0.011277  0.003305  0.019672
2006-12-31  0.052753  0.001471  0.025399  0.029361  0.015191
2007-12-31  0.082942  0.021521  0.035322  0.028592  0.016265
2008-12-31  0.016153  0.076571 -0.105776 -0.037095 -0.055838
2009-12-31  0.081402 -0.061191  0.085555  0.031111  0.037978
2010-12-31  0.113726  0.021065  0.026477  0.033278  0.007392
2011-12-31  0.043140  0.084843  0.006591  0.037461  0.028480
2012-12-31  0.031259  0.007426  0.039321  0.031366  0.043614
2013-12-31 -0.138713 -0.041927  0.074833  0.082597  0.116923
2014-12-31 -0.008912  0.077339  0.059762  0.057581  0.089859
2015-12-31 -0.042909 -0.005990  0.020370  0.027547  0.028039
2016-12-31  0.030147  0.003875  0.057792  0.020763 -0.011780
2017-12-31  0.050415  0.030606  0.143389  0.055656  0.091382
2018-12-31 -0.008204 -0.005691 -0.008243 -0.037034  0.029409
2019-12-

/var/folders/yh/nnw5blt935v54t02pj41_rjm0000gn/T/ipykernel_75686/979315640.py:5: FutureWarning: 'A' is deprecated and will be removed in a future version, please use 'YE' instead.
  annual_returns = daily_returns.resample('A').apply(lambda x: (x + 1).prod() - 1)


In [5]:
print(portfolio.stats())

Start                         2005-01-03 00:00:00
End                           2020-12-30 00:00:00
Period                         4027 days 00:00:00
Start Value                               10000.0
End Value                            17868.392434
Total Return [%]                        78.683924
Benchmark Return [%]                   384.922738
Max Gross Exposure [%]                  54.560759
Total Fees Paid                               0.0
Max Drawdown [%]                         15.63175
Max Drawdown Duration          1056 days 19:12:00
Total Trades                                  1.0
Total Closed Trades                           0.0
Total Open Trades                             1.0
Open Trade PnL                        7868.392434
Win Rate [%]                                  NaN
Best Trade [%]                                NaN
Worst Trade [%]                               NaN
Avg Winning Trade [%]                         NaN
Avg Losing Trade [%]                          NaN


/var/folders/yh/nnw5blt935v54t02pj41_rjm0000gn/T/ipykernel_75686/1076709553.py:1: UserWarning: Object has multiple columns. Aggregating using <function mean at 0x1159e5760>. Pass column to select a single column/group.
  print(portfolio.stats())
